# Tabular Modeling

Tabular deep NN models and collab models are very similar. Actually, they're identical! The only difference is that tabular models have more variables and possibly continuous variables.
- Collab filtering: (user,movie) → rating
- Tabular deep models: (user,age,gender,geo,income,movie,release,date,budget,director,language,genre) → rating

In [ ]:
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

# pd.options.display.max_rows = 20
# pd.options.display.max_columns = 8

# The Kaggle dataset

## Description

*Source: https://www.kaggle.com/c/bluebook-for-bulldozers/data*

The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations.

The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

Sample submission files can be downloaded from the data page. Submission files should be formatted as follows:

- Have a header: `SalesID,SalePrice`
- Contain two columns
  - `SalesID`: SalesID for the validation set in sorted order
  - `SalePrice`: Your predicted price of the sale

Example lines of the submission format:

`SalesID,SalePrice
1222837,36205
3044012,74570
1222841,31910.50
...`

## Data

View and download the benchmark code from Github: https://github.com/benhamner/BluebookForBulldozers/tree/master/Benchmark

For this competition, you are predicting the sale price of bulldozers sold at auctions.

The data for this competition is split into three parts:

- `Train.csv` is the training set, which contains data through the end of 2011.
- `Valid.csv` is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
- `Test.csv` is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.

The key fields are in train.csv are:

- `SalesID`: the uniue identifier of the sale
- `MachineID`: the unique identifier of a machine.  A machine can be sold multiple times
- `saleprice`: what the machine sold for at auction (only provided in train.csv)
- `saledate`: the date of the sale

There are several fields towards the end of the file on the different options a machine can have.  The descriptions all start with "machine configuration" in the data dictionary.  Some product types do not have a particular option, so all the records for that option variable will be null for that product type.  Also, some sources do not provide good option and/or hours data.

The machine_appendix.csv file contains the correct year manufactured for a given machine along with the make, model, and product class details. There is one machine id for every machine in all the competition datasets (training, evaluation, etc.).

## Downloading the data from Kaggle

In [ ]:
creds = '{"username":"rorymm","key":"690e177de576a42fe21868994c1b8093"}'

In [ ]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write(creds)
    cred_path.chmod(0o600)

In [ ]:
path = URLs.path('bluebook')
path

Path('/home/rory/.fastai/archive/bluebook')

In [ ]:
Path.BASE_PATH = path

In [ ]:
if not path.exists():
    path.mkdir()
    api.competition_download_cli('bluebook-for-bulldozers', path=path)
    file_extract(path/'bluebook-for-bulldozers.zip')

The above cell didn't work for me. Instead I ran in terminal:

`kaggle competitions download -c bluebook-for-bulldozers -p /home/rory/.fastai/archive/bluebook`

Then:
    
`file_extract(path/'bluebook-for-bulldozers.zip')`

In [ ]:
list(path.ls(file_type='text'))

[Path('Test.csv'),
 Path('median_benchmark.csv'),
 Path('ValidSolution.csv'),
 Path('Valid.csv'),
 Path('Machine_Appendix.csv'),
 Path('TrainAndValid.csv'),
 Path('random_forest_benchmark_test.csv')]

Let's load the raw data into a df.

In [ ]:
df = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)

## Inspecting the data

In [ ]:
df[0:3]

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
list(df.columns)

['SalesID',
 'SalePrice',
 'MachineID',
 'ModelID',
 'datasource',
 'auctioneerID',
 'YearMade',
 'MachineHoursCurrentMeter',
 'UsageBand',
 'saledate',
 'fiModelDesc',
 'fiBaseModel',
 'fiSecondaryDesc',
 'fiModelSeries',
 'fiModelDescriptor',
 'ProductSize',
 'fiProductClassDesc',
 'state',
 'ProductGroup',
 'ProductGroupDesc',
 'Drive_System',
 'Enclosure',
 'Forks',
 'Pad_Type',
 'Ride_Control',
 'Stick',
 'Transmission',
 'Turbocharged',
 'Blade_Extension',
 'Blade_Width',
 'Enclosure_Type',
 'Engine_Horsepower',
 'Hydraulics',
 'Pushblock',
 'Ripper',
 'Scarifier',
 'Tip_Control',
 'Tire_Size',
 'Coupler',
 'Coupler_System',
 'Grouser_Tracks',
 'Hydraulics_Flow',
 'Track_Type',
 'Undercarriage_Pad_Width',
 'Stick_Length',
 'Thumb',
 'Pattern_Changer',
 'Grouser_Type',
 'Backhoe_Mounting',
 'Blade_Type',
 'Travel_Controls',
 'Differential_Type',
 'Steering_Controls']

## Handle ordinal columns

Ordinal vars are discrete vars that have a specific order, such as "Small, Medium, Large".

In [ ]:
df['ProductSize'].unique()

array([nan, 'Medium', 'Small', 'Large / Medium', 'Mini', 'Large',
       'Compact'], dtype=object)

We have to manually order these by manually stating the order in a tuple, telling the df that the productsize column is a category, and then setting the categories using the tuple and the ordered=True param. Here's what that looks like:

In [ ]:
sizes = 'Large','Large / Medium','Medium','Small','Mini','Compact'

In [ ]:
df['ProductSize'] = df['ProductSize'].astype('category')

In [ ]:
df['ProductSize'].cat.set_categories(sizes, ordered=True, inplace=True)

## Handle dep var given the metric

Now that we've looked over all of the fields, it's time to think about the metric and the dep var.

The most important column in the dataset is the dep var (the one we want to train on and predict). In this step, we define the dep_var and apply any transformations to it that we need to given the metric.

In [ ]:
dep_var = 'SalePrice'

The competition says we're using RMSLE. (Picking the metric is a design process; however, this competition is telling us what to use, so we're skipping that design process which normally comes here). We're going to apply the log function to it now so we can use RMSE.

In [ ]:
df[dep_var] = np.log(df[dep_var])

## Enrich date fields

Now that we've looked over all the fields and handled the dep var, let's do some more data cleanup & enrichment. We can do some "feature engineering" to extract useful information out of this field. 

`add_datepart(df, 'date_col')`:
- adds 12 new columns to the end of the df
- removes date_col from the df

In [ ]:
df = add_datepart(df, 'saledate')

We had 53 columns before. Now we have 63:

In [ ]:
len(df.columns)

65

Let's take a look at what was added based on the saledate:

In [ ]:
list(df.columns)[(53-65):]

['saleYear',
 'saleMonth',
 'saleDay',
 'saleDayofweek',
 'saleDayofyear',
 'saleIs_month_end',
 'saleIs_month_start',
 'saleIs_quarter_end',
 'saleIs_quarter_start',
 'saleIs_year_end',
 'saleIs_year_start',
 'saleElapsed']

## Handle strings and missing data

We'll use fastai's class `TabularPandas`, which wraps a df and provides a few conveniences. To populate a TabularPandas object with data, we'll use two `TabularProcs`, `Categorify` and `FillMissing`. A TabularProc is like a regular Transform, except that:
- It returns the exact sam eobject that's passed to it after modifying it *in-place*
- It runs the transform once, when the data is first apssed in, rather than lazily as the data is accessed

`Categorify`:
- Replaces a column with a numeric categorical column

`FillMissing`:
- Replaces missing values with the median of the column
- Creates a new boolean columnn that is set to True for any row where the value was missing

These two transforms will be needed for nearly every dataset I will ever use.

In [ ]:
procs = [Categorify, FillMissing]

### Train/Valid split

We must be careful about how we construct our validation set. Because we're forecasting sales, we'll use the most recent data in the dataset as the validation set. We would NOT want to use random dates – we want to pick a point in time and predict the future from that point in time. To implement that, we'll use `np.where`.

TabularPandas will also handle the train/valid split.

In [ ]:
cond = (df.saleYear<2011) | (df.saleMonth<10)
train_idx = np.where(cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx), list(valid_idx))

In [ ]:
len(train_idx), len(valid_idx)

(404710, 7988)

## Cont/Cat vars

Time to tell TabularPandas which vars are continuous and which are categorical. We'll do this with the helper function `cont_cat_split`.

In [ ]:
### Bug fixes 2021-01-26 ###

## 1: cast saleWeek to int
# fix source: https://www.gitmemory.com/issue/fastai/fastai/3156/760969854
df.saleWeek = df.saleWeek.astype(int)

## 2: handle Ordinal type
# fix source: https://forums.fast.ai/t/problem-with-cont-cat-split-in-google-collab/84402/3
def cont_cat_split(df, max_card=20, dep_var=None):
    cont_names, cat_names = [], []
    for label in df:
        if label in L(dep_var): continue
        
        # mod to detect ProductSize type properly
        if (df[label].dtype.name == 'category'):
          cat_names.append(label)
          continue

        if (np.issubdtype(df[label].dtype, np.integer) and
            df[label].unique().shape[0] > max_card or
            np.issubdtype(df[label].dtype, np.floating)):
            cont_names.append(label)
        else: cat_names.append(label)
    return cont_names, cat_names

In [ ]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)
len(cont), len(cat)

(13, 51)

In [ ]:
tp = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

A `TabularPandas` `tp` object behaves a lot like a fastai `Datasets` object, incl `train` and `valid` attrs.

In [ ]:
len(tp.train), len(tp.valid)

(404710, 7988)

`tp.show` is a good way to viz the data for inspection.

In [ ]:
tp.show(3)

,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na,SalesID,MachineID,saleWeek,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saleYear,saleMonth,saleDay,saleDayofweek,saleDayofyear,SalePrice
0,Low,521D,521,D,#na#,#na#,#na#,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,False,False,False,False,1163635200,False,False,1139246,999089,46,3157,121,3.0,2004,68.0,2006,11,16,3,320,11.097410
1,Low,950FII,950,F,II,#na#,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,23.5,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,False,False,False,False,1080259200,False,False,1139248,117657,13,77,121,3.0,1996,4640.0,2004,3,26,4,86,10.950807
2,High,226,226,#na#,#na#,#na#,#na#,Skid Steer Loader - 1351.0 to 1601.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,#na#,OROPS,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Auxiliary,#na#,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,None or Unspecified,Standard,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,False,False,False,False,1077753600,False,False,1139249,434808,9,7009,121,3.0,2001,2838.0,2004,2,26,3,57,9.210340


Compare the above to the "real" data, which is all numeric. It's accessible through `tp.items`:

In [ ]:
tp.items.head(3)

,SalesID,SalePrice,MachineID,saleWeek,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,...,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,11.097410,999089,46,3157,121,3.0,2004,68.0,2,...,320,1,1,1,1,1,1,2647,1,1
1,1139248,10.950807,117657,13,77,121,3.0,1996,4640.0,2,...,86,1,1,1,1,1,1,2148,1,1
2,1139249,9.210340,434808,9,7009,121,3.0,2001,2838.0,1,...,57,1,1,1,1,1,1,2131,1,1


To lookup a numeric value to find its 'vocab', we use tp.classes[\'col_name\']:

In [ ]:
tp.classes['ProductSize']

['#na#', 'Large', 'Large / Medium', 'Medium', 'Small', 'Mini', 'Compact']

In [ ]:
### LEFT OFF HERE ###
# https://course.fast.ai/videos/?lesson=7
# 50:29

# Decision Trees

Here's the approach:
1. Loop through each column in the dataset
2. Loop through each possible value of the column
3. Split the data into two groups based on each value
4. Find the average sale price for each group
5. After doing this for all cols and all vals, we choose the split with the best RMSE as our first binary decision

Basically we've created a model that simply splits the entire dataset into two groups using one single column.

6. The dataset is now split into two groups after the first decision. For each child group, repeat steps 1-5.
7. Continue this process recursively until we've reached a stopping criterion, such as a minimum sample size per group.